In [ ]:
from mainnet_launch.autopool_diagnostics.compute_rebalance_cost import (
    get_data,
    AUTO_LRT,
    BAL_ETH,
    AUTO_ETH,
    eth_client,
    AutopoolConstants,
    _fetch_all_rebalance_events
)
import pandas as pd
import plotly.express as px

import plotly.io as pio
from mainnet_launch.destinations import get_destination_details, ALL_AUTOPOOLS

pio.templates.default = None
autopool = AUTO_LRT

def make_actual_swap_cost_df(autopool) -> pd.DataFrame
    rebalance_between_destinations_df, validated_spot_price_df, UnderlyingDeposited_df = get_data(autopool)

    valid_deposit_hashes = UnderlyingDeposited_df['hash'].value_counts()[UnderlyingDeposited_df['hash'].value_counts() == 1].index
    valid_underlying_deposited_df=UnderlyingDeposited_df[UnderlyingDeposited_df['hash'].isin(valid_deposit_hashes)].copy()
    valid_underlying_deposited_df['amountIn'] = valid_underlying_deposited_df['amount'] / 1e18
    df = pd.merge(rebalance_between_destinations_df, valid_underlying_deposited_df, on='hash', how='inner')
    df['raw_amount_out'] = df["params"].apply(lambda x: x[5])
    df['raw_amount_in'] = df["amount"]
    df['event_amount_in'] = df["params"].apply(lambda x: x[2])
    df['block'] = df['block_y']

    limited_df = df[['contract_address', 'amountIn', 'amountOut', 'inDestinationVault', 'outDestinationVault',
                    'hash', 'block', 'raw_amount_in', 'raw_amount_out', 'event_amount_in', 'swapCost']].copy()


    amounts_with_spot_values = pd.merge(limited_df, validated_spot_price_df, on='block', how='inner')
    def _compute_spot_value_change(row):
        in_dest = row['inDestinationVault']
        in_dest_spot_price = row[in_dest]
        
        out_dest = row['outDestinationVault']
        out_dest_spot_price = row[out_dest]
        
        spot_value_out = out_dest_spot_price * row['amountOut']
        spot_value_in = in_dest_spot_price * row['amountIn']

        swap_cost = spot_value_out - spot_value_in

        return {
            'spot_value_out':spot_value_out,
            'spot_value_in':spot_value_in,
            'spot_value_swap_cost': swap_cost
        }
        
    spot_values_df = pd.DataFrame.from_records(amounts_with_spot_values.apply(_compute_spot_value_change, axis=1).values)
    amounts_with_spot_values = pd.concat([amounts_with_spot_values, spot_values_df], axis=1)
    return amounts_with_spot_values # spot_value_swap_cost and # destination In to split it

2024-11-13 21:17:15.284 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 21:17:15.288 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 21:17:15.292 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 21:17:15.294 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 21:17:15.296 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 21:17:15.297 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 21:17:15.299 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 21:17:15.300 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-

In [ ]:


px.line(amounts_with_spot_values[['swapCost', 'spot_value_swap_cost']].cumsum())

,contract_address,amountIn,amountOut,inDestinationVault,outDestinationVault,hash,block,raw_amount_in,raw_amount_out,event_amount_in,...,0x90300b02b162F902B9629963830BcCCdeEd71113,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0x17216C32B9134343258B05807405db02BE030acd,0x689dc7B5CE8e23061a8223E97701E9345764B8B7,0x676B34c2C2294eF070c1e9E0FCD4091Dd06393FE,0x1416BfE53520a71553230c3FdeeEce30d5470bA7,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0x40219bBda953ca811d2D0168Dc806a96b84791d9,0x148Ca723BefeA7b021C399413b8b7426A4701500,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C
0,0x777FAf85c8E5FC6f4332E56B989C5C94201A273C,100.512375,103.024424,0x777FAf85c8E5FC6f4332E56B989C5C94201A273C,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x5a6f3a2239134ef903468d88abce973e6b71e233af04...,20767364,100512374560900536081,103024423626862362624,100512272441216991232,...,1.016436,NaN,1.024542,1.014292,1.013827,1.016436,1.016099,NaN,1.018699,NaN
1,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,103.715831,105.124100,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x46cfba2d4f806b97e71d9465ae05556bc41cb3635c5d...,20772145,103715830557886143959,105124100397257129984,103572485503304712192,...,1.015869,NaN,1.024548,1.014539,1.013652,1.015869,1.016070,NaN,1.019026,NaN
2,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,165.884649,168.718684,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xa0ea869df690411f31e8a430670cfaea2b69a5849715...,20778301,165884648743842835763,168718684271991422976,165744495431202996224,...,1.016686,NaN,1.025236,1.014812,1.013926,1.016686,1.016337,NaN,1.019406,NaN
3,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,188.896729,191.505927,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xca793ddcca9f1a355bf52d55d397068f49b8c43de799...,20781400,188896729234516955929,191505927169537048576,188630309468137127936,...,1.016768,NaN,1.025257,1.014962,1.013939,1.016768,1.016391,NaN,1.019393,NaN
4,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,141.824778,144.375381,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x6ce994c065fa58c7b58d215373dcb7480aba63965013...,20786072,141824778293424392382,144375380803665870848,141824778070793191424,...,1.016107,NaN,1.024948,1.015160,1.013764,1.016107,1.016182,NaN,1.019049,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,98.265258,100.000000,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x7ea296cc9d700bc8b3a817e36f343928d997bd65d3b7...,21141208,98265257997978557172,100000000000000000000,98102479908599791616,...,1.018914,1.017368,1.030730,1.018058,1.017368,1.018914,1.019286,1.020552,1.020552,1.018914
76,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,98.280908,100.000000,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x06c28f98ff45d82f34a0034733773a6f10c8e80fdced...,21143968,98280908050360264648,100000000000000000000,98119584275799883776,...,1.018568,1.017422,1.030700,1.018410,1.017422,1.018568,1.019382,1.019294,1.019294,1.018568
77,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,97.770709,99.483984,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x84da6e052ce802cd1dc77ead5d40eb124a14c4aefc46...,21148318,97770709315536396959,99483984153862750208,97613489560750473216,...,1.017863,1.017375,1.030671,1.017331,1.017375,1.017863,1.019488,1.019870,1.019870,1.017863
78,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,73.300808,74.665379,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xd899eb779a909d345dac9d56abbf0a842b5fe4c5914f...,21152799,73300808373982903832,74665379379959873536,73265691609130049536,...,1.017749,1.017412,1.030697,1.017328,1.017412,1.017749,1.019411,1.019175,1.019175,1.017749


,contract_address,amountIn,amountOut,inDestinationVault,outDestinationVault,hash,block,raw_amount_in,raw_amount_out,event_amount_in,...,0x689dc7B5CE8e23061a8223E97701E9345764B8B7,0x676B34c2C2294eF070c1e9E0FCD4091Dd06393FE,0x1416BfE53520a71553230c3FdeeEce30d5470bA7,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0x40219bBda953ca811d2D0168Dc806a96b84791d9,0x148Ca723BefeA7b021C399413b8b7426A4701500,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C,spot_value_out,spot_value_in,spot_value_swap_cost
0,0x777FAf85c8E5FC6f4332E56B989C5C94201A273C,100.512375,103.024424,0x777FAf85c8E5FC6f4332E56B989C5C94201A273C,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x5a6f3a2239134ef903468d88abce973e6b71e233af04...,20767364,100512374560900536081,103024423626862362624,100512272441216991232,...,1.014292,1.013827,1.016436,1.016099,NaN,1.018699,NaN,103.024424,102.979160,0.045264
1,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,103.715831,105.124100,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x46cfba2d4f806b97e71d9465ae05556bc41cb3635c5d...,20772145,103715830557886143959,105124100397257129984,103572485503304712192,...,1.014539,1.013652,1.015869,1.016070,NaN,1.019026,NaN,105.124100,105.131775,-0.007675
2,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,165.884649,168.718684,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xa0ea869df690411f31e8a430670cfaea2b69a5849715...,20778301,165884648743842835763,168718684271991422976,165744495431202996224,...,1.014812,1.013926,1.016686,1.016337,NaN,1.019406,NaN,168.718684,168.594627,0.124057
3,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,188.896729,191.505927,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xca793ddcca9f1a355bf52d55d397068f49b8c43de799...,20781400,188896729234516955929,191505927169537048576,188630309468137127936,...,1.014962,1.013939,1.016768,1.016391,NaN,1.019393,NaN,191.505927,191.529813,-0.023886
4,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,141.824778,144.375381,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x6ce994c065fa58c7b58d215373dcb7480aba63965013...,20786072,141824778293424392382,144375380803665870848,141824778070793191424,...,1.015160,1.013764,1.016107,1.016182,NaN,1.019049,NaN,144.375381,144.119784,0.255597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,98.265258,100.000000,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x7ea296cc9d700bc8b3a817e36f343928d997bd65d3b7...,21141208,98265257997978557172,100000000000000000000,98102479908599791616,...,1.018058,1.017368,1.018914,1.019286,1.020552,1.020552,1.018914,100.000000,99.971889,0.028111
76,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,98.280908,100.000000,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x06c28f98ff45d82f34a0034733773a6f10c8e80fdced...,21143968,98280908050360264648,100000000000000000000,98119584275799883776,...,1.018410,1.017422,1.018568,1.019382,1.019294,1.019294,1.018568,100.000000,99.993196,0.006804
77,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,97.770709,99.483984,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x84da6e052ce802cd1dc77ead5d40eb124a14c4aefc46...,21148318,97770709315536396959,99483984153862750208,97613489560750473216,...,1.017331,1.017375,1.017863,1.019488,1.019870,1.019870,1.017863,99.483984,99.469439,0.014545
78,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,73.300808,74.665379,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xd899eb779a909d345dac9d56abbf0a842b5fe4c5914f...,21152799,73300808373982903832,74665379379959873536,73265691609130049536,...,1.017328,1.017412,1.017749,1.019411,1.019175,1.019175,1.017749,74.665379,74.577089,0.088291


In [ ]:
break

In [7]:
amounts_with_spot_values.columns

Index(['contract_address', 'amountIn', 'amountOut', 'inDestinationVault',
       'outDestinationVault', 'hash', 'block', 'raw_amount_in',
       'raw_amount_out', 'event_amount_in', 'swapCost',
       '0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126',
       '0x3B2C87Ed95483059b950155a68E906d543193d5C',
       '0xE800e3760FC20aA98c5df6A9816147f190455AF3',
       '0x777FAf85c8E5FC6f4332E56B989C5C94201A273C',
       '0x2f2CC1bf461413014741dD68481dB4a3686DAC3D',
       '0xB6d68122428Dc1141467cB96791618615Ab9F746',
       '0x60339056EC88996e41757E05a798310E46972cca',
       '0xC4c973eDC82CB6b972C555672B4e63713C177995',
       '0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575',
       '0x90300b02b162F902B9629963830BcCCdeEd71113',
       '0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc',
       '0x17216C32B9134343258B05807405db02BE030acd',
       '0x689dc7B5CE8e23061a8223E97701E9345764B8B7',
       '0x676B34c2C2294eF070c1e9E0FCD4091Dd06393FE',
       '0x1416BfE53520a71553230c3FdeeEce30d5470bA7',
       '0x2

In [8]:
rebalances_we_can_compute_swap_costs_for = rebalance_between_destinations_df[rebalance_between_destinations_df['hash'].isin(valid_underlying_deposited_df['hash'])].copy()

rebalances_we_can_compute_swap_costs_for[['outDestinationVault', 'hash', 'amountOut']]

,outDestinationVault,hash,amountOut
0,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x5a6f3a2239134ef903468d88abce973e6b71e233af04...,103.024424
1,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x46cfba2d4f806b97e71d9465ae05556bc41cb3635c5d...,105.124100
2,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xa0ea869df690411f31e8a430670cfaea2b69a5849715...,168.718684
3,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xca793ddcca9f1a355bf52d55d397068f49b8c43de799...,191.505927
4,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x6ce994c065fa58c7b58d215373dcb7480aba63965013...,144.375381
...,...,...,...
78,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x7ea296cc9d700bc8b3a817e36f343928d997bd65d3b7...,100.000000
79,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x06c28f98ff45d82f34a0034733773a6f10c8e80fdced...,100.000000
80,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x84da6e052ce802cd1dc77ead5d40eb124a14c4aefc46...,99.483984
81,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xd899eb779a909d345dac9d56abbf0a842b5fe4c5914f...,74.665379


In [9]:
114 -

SyntaxError: invalid syntax (3529762651.py, line 1)

In [ ]:
valid_underlying_deposited_df

In [ ]:
valid_underlying_deposited_df['amount']

In [ ]:
valid_underlying_deposited_df['contract_address'].value_counts()

In [ ]:
hashes_to_exclude = UnderlyingDeposited_df['hash'].value_counts()[UnderlyingDeposited_df['hash'].value_counts() > 1].index
# sunseting destiantions throws the math off and has 0 slippage anyway
hashes_to_exclude

In [ ]:
# 0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005    2
# 0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b    2
# 0xa9b7e57b8202064b6fd48accbcfea54cb08fbb362638aa47681203c5daf7b722    2
UnderlyingDeposited_df[UnderlyingDeposited_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

In [ ]:
UnderlyingWithdraw_df[UnderlyingWithdraw_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

In [ ]:
rebalance_between_destinations_df.columns

In [ ]:
cols = ['hash', 'amountOut', 'outDestinationVault']
# 0xE800e3760FC20aA98c5df6A9816147f190455AF3 == autoLRT vault, all WETH
rebalance_between_destinations_df[cols]['outDestinationVault'].value_counts()

In [ ]:
UnderlyingDeposited_df['hash'].value_counts()

In [ ]:
# amount 

In [ ]:
rebalance_between_destinations_df['hash'].value_counts()

In [ ]:
RebalanceBetweenDestinations, RebalanceToIdle, SuccessfulRebalanceBetweenDestinations = dfs
RebalanceBetweenDestinations.shape, RebalanceToIdle.shape, SuccessfulRebalanceBetweenDestinations.shape

In [ ]:
RebalanceToIdle["tokenIn"] = RebalanceToIdle["params"].apply(
    lambda x: eth_client.toChecksumAddress(x[1])
)

RebalanceToIdle["tokenOut"] = RebalanceToIdle["params"].apply(
    lambda x: eth_client.toChecksumAddress(x[4])
)

RebalanceToIdle["amountOut"] = RebalanceToIdle["params"].apply(
    lambda x: x[5]
    / 1e18  # the amout of lp tokens taken out is always correct, what we don't know is the amount of lp tokens going in
    )
RebalanceToIdle

In [ ]:
RebalanceToIdle['hash'].values

In [ ]:
RebalanceToIdle['hash'].isin(RebalanceBetweenDestinations['hash'])

In [ ]:
SuccessfulRebalanceBetweenDestinations['hash'].isin(RebalanceBetweenDestinations['hash'])

In [ ]:
0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005    2
0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b    2
0xa9b7e57b8202064b6fd48accbcfea54cb08fbb362638aa47681203c5daf7b722    2
UnderlyingDeposited_df[UnderlyingDeposited_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

In [ ]:
UnderlyingDeposited_df[UnderlyingDeposited_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

In [ ]:
rebalance_between_destinations_df[rebalance_between_destinations_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

In [ ]:
UnderlyingDeposited_df['hash'].value_counts()

In [ ]:
def underlying_deposited_df_to_change(row):
    return {
        'Destination': row['contract_address'],
        'lpTokenQuantity': row['amount'] /1e18,
        'hash':row['hash'],
        'block': int(row['block'])
    }

deposit_df = pd.DataFrame.from_records(UnderlyingDeposited_df.apply(underlying_deposited_df_to_change, axis=1).values)

def underlying_withdrawn_df_to_change(row):
    return {
        'Destination': row['contract_address'],
        'lpTokenQuantity':  - row['amount'] /1e18,
        'hash':row['hash'],
        'block': int(row['block'])
    }

withdraw_df = pd.DataFrame.from_records(UnderlyingWithdraw_df.apply(underlying_withdrawn_df_to_change, axis=1).values)
change_df = pd.concat([deposit_df, withdraw_df]).sort_values('block')
change_df

In [ ]:
# AMOUNT_OUT IS THE QUANITY OF LP TOKENS THAT LEFT
# WE JUST NEED THE DEPOSTED

In [ ]:
df = pd.merge(change_df, validated_spot_price_df, on='block')

def _change_in_spot_value(row):
    spot_price =  row[row['Destination']]
    change = spot_price * row['lpTokenQuantity']
    return change

df['spot_value_change'] = df.apply(_change_in_spot_value, axis=1)

a = df.groupby('hash')['spot_value_change'].apply(list)

between_destination_hashes = a[a.apply(len) == 2].index
from_idle_hashes = a[a.apply(len) == 1].index

In [ ]:
amounts_and_quantities = pd.merge(withdraw_df[['hash', 'lpTokenQuantity']], rebalance_between_destinations_df[['amountOut', 'hash']], on='hash')
amounts_and_quantities[amounts_and_quantities['hash'].isin(between_destination_hashes)]

In [ ]:
amounts_and_quantities = pd.merge(df[['hash', 'lpTokenQuantity']], rebalance_between_destinations_df[['amountOut', 'hash']], on='hash')
amounts_and_quantities[amounts_and_quantities['hash'].isin(from_idle_hashes)]

In [ ]:
rebalance_between_destinations_df[['amountOut', 'hash']]

In [ ]:
rebalance_between_destinations_df[rebalance_between_destinations_df['hash'].isin(between_destination_hashes)][['outDestinationVault', 'inDestinationVault']].value_counts()

In [ ]:
df.groupby('hash')['spot_value_change'].apply(len) == 2

In [ ]:
UnderlyingWithdraw_df

In [ ]:


# UnderlyingDeposited_df["amountDeposited"] = UnderlyingDeposited_df["amount"] / 1e18
# UnderlyingWithdraw_df["amountWithdraw"] = UnderlyingWithdraw_df["amount"] / 1e18

# df = pd.merge(rebalance_between_destinations_df, UnderlyingDeposited_df[['hash', 'amountDeposited']], how='left', on='hash')
# df = pd.merge(df, UnderlyingWithdraw_df[['hash', 'amountWithdraw']], how='left', on='hash')
# cols = ['hash', 'amountWithdraw', 'amountDeposited', 'amountOut', 'tokenIn', 'tokenOut']

In [ ]:
# df['hash'].value_counts() # exclude the funny ones, with destinations getting added or withdrawn

In [ ]:
df[df["hash"] == "0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005"][cols]

In [ ]:
UnderlyingDeposited_df[
    UnderlyingDeposited_df["hash"] == "0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005"
]

In [ ]:
UnderlyingWithdraw_df[
    UnderlyingWithdraw_df["hash"] == "0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005"
]

In [ ]:
df2 = df.dropna()
df2

In [ ]:
WETH = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
#  emit UnderlyingDeposited(amount, msg.sender);

In [ ]:
rebalance_between_destinations_df[["tokenIn", "tokenOut", "amountOut", "destinationIn"]]

In [ ]:
rebalance_between_destinations_df["hash"].values

In [ ]:
first_out = "0xE800e3760FC20aA98c5df6A9816147f190455AF3"
first_in = "0x777FAf85c8E5FC6f4332E56B989C5C94201A273C"
balance_of_undelrying_debt_before[[first_in, first_out]]
# I can't check the balances before or after

In [ ]:
balance_of_undelrying_debt_before[[first_in, first_out]]

In [ ]:
balance_of_undelrying_debt_after[[first_in, first_out]]

In [ ]:
rebalance_between_destinations_df

In [ ]:
rebalance_between_destinations_df

In [ ]:
validated_spot_price_df

In [ ]:
# from mainnet_launch.autopool_diagnostics.compute_rebalance_cost import (
#     compute_daily_rebalance_costs,
#     AUTO_LRT, BAL_ETH, AUTO_ETH,
#     AutopoolConstants,
# )
# import pandas as pd
# import plotly.express as px


# # # return RebalanceValueStats({
# # #     inPrice: inPrice,
# # #     outPrice: outPrice,
# # #     inEthValue: inEthValue,
# # #     outEthValue: outEthValue,
# # #     swapCost: swapCost,
# # #     slippage: slippage
# # # });


# # /// @param destinationIn The address / lp token of the destination vault that will increase
# # /// @param tokenIn The address of the underlyer token that will be provided by the swapper
# # /// @param amountIn The amount of the underlying LP tokens that will be received
# # /// @param destinationOut The address of the destination vault that will decrease
# # /// @param tokenOut The address of the underlyer token that will be received by the swapper
# # /// @param amountOut The amount of the tokenOut that will be received by the swapper
# # struct RebalanceParams {
# #     address destinationIn;
# #     address tokenIn;
# #     uint256 amountIn;
# #     address destinationOut;
# #     address tokenOut;
# #     uint256 amountOut;
# # }
# autopool = AUTO_ETH
# a, b = compute_daily_rebalance_costs(autopool)

In [ ]:
autopool = AUTO_ETH

In [ ]:
import plotly.io as pio
from mainnet_launch.destinations import get_destination_details, ALL_AUTOPOOLS

pio.templates.default = None
rebalance_from_idle, transfer_events_df, validated_spot_price_df = b
rebalance_between_destinations, transfer_events_df, validated_spot_price_df = a

In [ ]:
desination_vault_addresses = [d.vaultAddress for d in get_destination_details() if d.autopool == autopool]
desination_vault_addresses

In [ ]:
def _get_limited_transfers(
    autopool: AutopoolConstants, transfer_events_df: pd.DataFrame, rebalance_df: pd.DataFrame
) -> pd.DataFrame:
    desination_vault_addresses = [d.vaultAddress for d in get_destination_details() if d.autopool == autopool]
    sender_or_receiver_is_destination_vault = (transfer_events_df["to"].isin(desination_vault_addresses)) | (
        transfer_events_df["from"].isin(desination_vault_addresses)
    )
    destination_lp_token_addresses = [d.vaultAddress for d in get_destination_details() if d.autopool == autopool]
    sent_token_is_lp_token_avaiable_to_autopool = transfer_events_df["token_address"].isin(
        destination_lp_token_addresses
    )

    tx_hash_in_rebalance_df_hash = transfer_events_df["hash"].isin(rebalance_df["hash"])

    limited_transfers = (
        transfer_events_df[
            sender_or_receiver_is_destination_vault
            & sent_token_is_lp_token_avaiable_to_autopool
            & tx_hash_in_rebalance_df_hash
        ]
        .copy()
        .reset_index()
    )

    return limited_transfers


limited_transfers = _get_limited_transfers(autopool, transfer_events_df, rebalance_between_destinations)
limited_transfers["hash"].value_counts()

In [ ]:
# def compute_swap_cost_when_rebalance_from_idle(
#     rebalance_from_idle_df: pd.DataFrame, autopool: AutopoolConstants, transfer_events_df: pd.DataFrame, validated_spot_price_df:pd.DataFrame
# ) -> pd.DataFrame:
#     # out means decrease
#     # in means increase
#     if (rebalance_from_idle_df["outDestinationVault"] != autopool.autopool_eth_addr).any():
#         raise ValueError(
#             "trying to compute rebalance cost on rebalance_from_idle_df when the outDestinationVault != autopool "
#         )


#     #

#     transfers_to_vaults = (
#         transfers_to_or_from_autopool_destinations[
#             (transfer_events_df["hash"].isin(rebalance_from_idle_df["hash"]))
#             & (transfer_events_df["to"].isin(rebalance_from_idle_df["inDestinationVault"]))
#         ]

#         .drop_duplicates()
#         .copy()
#     )

#     rebalance_from_idle_with_lp_tokens_in = pd.merge(
#         rebalance_from_idle_df, transfers_to_vaults[["value", "hash"]], on="hash"
#     )

#     rebalance_from_idle_with_lp_tokens_in["lp_tokens_added_to_vault"] = (
#         rebalance_from_idle_with_lp_tokens_in["value"]/ 1e18
#     )

#     validated_spot_price_df["block_with_rebalance"] = ( validated_spot_price_df["block"] + 1)

#     rebalance_from_idle_df = pd.merge(
#         validated_spot_price_df,
#         rebalance_from_idle_with_lp_tokens_in,
#         right_on="block",
#         left_on="block_with_rebalance",
#     )

#     def _compute_spot_value_added_to_vault(row:dict) -> float:
#         return row['lp_tokens_added_to_vault'] * row[row['inDestinationVault']] # the pool itsel

#     rebalance_from_idle_df['validated_spot_value_of_lp_tokens_added_to_autopool'] = rebalance_from_idle_df.apply(_compute_spot_value_added_to_vault, axis=1)
#     # because we are exiting idle, the value out is in WETH so the spot value is always 1
#     rebalance_from_idle_df['validated_spot_value_of_lp_tokens_removed_from_autopool'] = rebalance_from_idle_df["params_amountOut"]
#     rebalance_from_idle_df['validated_spot_value_of_swap_cost'] = rebalance_from_idle_df['validated_spot_value_of_lp_tokens_removed_from_autopool'] -  rebalance_from_idle_df['validated_spot_value_of_lp_tokens_added_to_autopool']

#     rebalance_from_idle_df['validated_spot_value_of_swap_cost'] = rebalance_from_idle_df['validated_spot_value_of_swap_cost'].clip(lower=0)

#     rebalance_from_idle_df['swap_cost_from_event'] = rebalance_from_idle_df['valueStats'].apply(lambda x: float(x[4]) /1e18)

#     # not sure what do to with the swap cost
#     rebalance_from_idle_df.index = validated_spot_price_df.index
#     cumulative_daily_validated_swap_cost =  rebalance_from_idle_df[['validated_spot_value_of_swap_cost']].resample("1D").sum().cumsum()
#     return cumulative_daily_validated_swap_cost

# rebalance_events_df, transfer_events_df, validated_spot_price_df = b
# cumulative_daily_validated_swap_cost = compute_swap_cost_when_rebalance_from_idle(
#     rebalance_events_df.copy(), AUTO_ETH, transfer_events_df, validated_spot_price_df
# )

# cumulative_daily_validated_swap_cost

In [ ]:
# rebalance_between_destinations_df, transfer_events_df, validated_spot_price_df = a

# def _extract_relevent_transfers(autopool: AutopoolConstants, transfer_events_df: pd.DataFrame, rebalance_between_destinations_df:pd.DataFrame):

#     valid_destinations = [d.vaultAddress for d in get_destination_details() if d.autopool == autopool]
#     valid_transfer_events = transfer_events_df[(transfer_events_df['to'].str.lower().isin(valid_destinations)) | (transfer_events_df['from'].str.lower().isin(valid_destinations))]

#     to_indest = (
#         transfer_events_df[
#             (transfer_events_df["hash"].isin(rebalance_between_destinations_df["hash"]))
#             & (transfer_events_df["to"].isin(valid_destinations))
#         ]
#         .drop_duplicates()
#         .copy()
#     )

#     to_outdest = (
#         transfer_events_df[
#             (transfer_events_df["hash"].isin(rebalance_between_destinations_df["hash"]))
#             & (transfer_events_df["to"].isin(valid_destinations))
#         ]

#         .drop_duplicates()
#         .copy()
#     )


#     from_indest = (
#         transfer_events_df[
#             (transfer_events_df["hash"].isin(rebalance_between_destinations_df["hash"]))
#             & (transfer_events_df["from"].isin(valid_destinations))
#         ]
#         .drop_duplicates()
#         .copy()
#     )

#     from_outdest = (
#         transfer_events_df[
#             (transfer_events_df["hash"].isin(rebalance_between_destinations_df["hash"]))
#             & (transfer_events_df["from"].isin(valid_destinations))
#         ]

#         .drop_duplicates()
#         .copy()
#     )


#     return to_indest, to_outdest, from_indest, from_outdest

# to_indest, to_outdest, from_indest, from_outdest = _extract_relevent_transfers(AUTO_ETH, transfer_events_df, rebalance_between_destinations_df)
# to_indest.shape, to_outdest.shape, from_indest.shape, from_indest.shape,

In [ ]:
between_with_from_indest = pd.merge(rebalance_between_destinations_df, from_indest[["value", "hash"]], on="hash")

between_with_from_indest["hash"].value_counts()

In [ ]:
rebalance_between_destinations_df[
    rebalance_between_destinations_df["hash"] == "0x5a088e71e51c5b8c05db282989f21975d14e44d9c615aa00e451a68dcbebd0dc"
]

In [ ]:
# 0x5a088e71e51c5b8c05db282989f21975d14e44d9c615aa00e451a68dcbebd0dc rebalances more than one block

In [ ]:
transfers_from_vaults[
    transfers_from_vaults["hash"] == "0x5a088e71e51c5b8c05db282989f21975d14e44d9c615aa00e451a68dcbebd0dc"
]

In [ ]:
transfers_from_vaults["hash"].value_counts()

In [ ]:
# # def compute_swap_cost_when_rebalancing_between_destinations(
# #     rebalance_not_from_idle_df: pd.DataFrame, autopool: AutopoolConstants, transfer_events_df: pd.DataFrame, validated_spot_price_df:pd.DataFrame
# # ) -> pd.DataFrame:
# #     # out means decrease
# #     # in means increase
# #     if (rebalance_not_from_idle_df["outDestinationVault"] == autopool.autopool_eth_addr).any():
# #         raise ValueError(
# #             "trying to compute_swap_cost_when_rebalancing_between_destinations when the outDestinationVault == autopool eg, exiting the autopool "
# #         )


# #     limited_transfer_df = (
# #         transfer_events_df[
# #             (transfer_events_df["hash"].isin(rebalance_not_from_idle_df["hash"]))
# #         ]

# #         .drop_duplicates()
# #         .copy()
# #     )
# #     return limited_transfer_df


# #

# # limited_transfer_df = compute_swap_cost_when_rebalancing_between_destinations(
# #     rebalance_events_df.copy(), AUTO_ETH, transfer_events_df, validated_spot_price_df
# # )

# # transfers_to_vaults['to_value'] = transfers_to_vaults['value'] /1e18
# # transfers_to_vaults['to_token'] = transfers_to_vaults['token_address']

# # transfers_from_vaults['from_value'] = transfers_from_vaults['value'] /1e18
# # transfers_from_vaults['from_token'] = transfers_from_vaults['token_address']

# def build_lp_token_moved_df(row):

#     return {
#         'DesinationIncreased': row['to_token'],
#         'DestinationDecreased': row['from_token'],
#         'amountIncreased': row['to_value'],
#         'amountDecreased': row['from_value'],
#         'hash':row['hash'],
#         'block': row['block']
#     }

# limited_transfer_df['hash'].value_counts()

In [ ]:
rebalance_events_df

In [ ]:
transfers_from_vaults

In [ ]:
rebalance_events_df["inDestinationVault"].value_counts()

In [ ]:
rebalance_events_df["outDestinationVault"].value_counts()

In [ ]:
AUTO_ETH.autopool_eth_addr

In [ ]:
rebalance_from_idle_df.sort_values("rebalance_cost")[["hash", "rebalance_cost"]].values[0]

0xf06978787ae75d235cedd61c07b1e2a9037df86378f37a90eb683c87db8f98b2

This rebalance, my math thinks that

In [ ]:
rebalance_from_idle_df["valueStats_swap_cost"] = rebalance_from_idle_df["valueStats"].apply(lambda x: x[4] / 1e18)
px.line(rebalance_from_idle_df[["rebalance_cost", "valueStats_swap_cost"]])

In [ ]:
dest_to_dest = lp_token_transfer_df[(lp_token_transfer_df["hash"] == tx_hash)].copy().drop_duplicates()
dest_to_dest

In [ ]:
# 165151197070039380229 expect outDestinationLpToken to be down

today I am working more on fixing the overestimate the rebalancing cost on the dashboard. 

In [ ]:
# idle -> destination valueStats.amountOut == quantiy of eth that left
#

In [ ]:
rebalance_events_df.shape

In [ ]:
160795862717038788608 + 419684887152871954

In [ ]:
dest_to_dest[dest_to_dest["token_address"] == inDestinationVault]

In [ ]:
dest_to_dest[dest_to_dest["token_address"] == outDestinationVault]

In [ ]:
dest_to_dest[dest_to_dest["to"] == outDestinationVault]

In [ ]:
dest_to_dest[dest_to_dest["from"] == outDestinationVault]
# destination vault -> flash borrow solver 0x067cB94576aBBDB3527055f3c8CaAAc9668364AD
# is the quanikty t

In [ ]:
dest_to_dest["to"].value_counts()

In [ ]:
rebalance_events_df.values[-1]  # 73804358107532979104

In [ ]:
rebalance_events_df